In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sklearn
print("sklearn: ",sklearn.__version__)
import sys
print("Python: ",sys.version)

sklearn:  0.21.2
Python:  3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

print(tf.VERSION)
print(tf.keras.__version__)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import scipy.io
from PIL import Image

plt.ion()   # interactive mode

In [ ]:
!nvidia-smi

In [ ]:
if torch.cuda.is_available():
    devID=torch.cuda.current_device()
    print("GPU: ",torch.cuda.get_device_name(devID))
else:
    print("Torch Cuda not avaialbe")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
torch.cuda.is_available()

In [ ]:
PATH = Path('../../../../../data/StanfordCars')

In [ ]:
import os
print(os.listdir(PATH))

In [ ]:
PATH.ls()

In [ ]:
path_train = PATH/'cars_train'
path_test = PATH/'cars_test'

In [ ]:
fnames = get_image_files(path_train)
fnames[:5]

In [ ]:
print(f'Total number of images: {len(fnames)}')

In [ ]:
pic = Path(f'{path_train}/00001.jpg')
img = plt.imread(pic)
plt.imshow(img);

In [ ]:
print(os.listdir(f'{PATH}/devkit'))

In [ ]:
#load cars class labels matlab file
class_annos = scipy.io.loadmat(PATH/'devkit/cars_meta.mat')

In [ ]:
class_annos

In [ ]:
class_annos["class_names"][0][14][0]

In [ ]:
#load train annotiation file
train_annos = scipy.io.loadmat(PATH/'devkit/cars_train_annos.mat')

In [ ]:
# matlab file decoding mask inside 'annotations' array
# dtype=[('bbox_x1', 'O'), ('bbox_y1', 'O'), ('bbox_x2', 'O'), ('bbox_y2', 'O'), ('class', 'O'), ('fname', 'O')])}
train_annos

In [ ]:
# extract item 0 class
train_annos["annotations"][0][0][4][0][0]

In [ ]:
# extract itme 0 file name
train_annos["annotations"][0][0][5][0]

In [ ]:
total_set = len(train_annos["annotations"][0])
print(f'Total number of data items in dataset: {total_set}')

In [ ]:
classes_total = len(class_annos["class_names"][0])
print(f'Total number of classes: {classes_total}')

In [ ]:
# loop to extract file name and class number into list
#                                                     fname                                    class
classes = [[f'cars_train/{train_annos["annotations"][0][i][5][0]}',train_annos["annotations"][0][i][4][0][0]] for i in range(total_set)]
classes[:5]

In [ ]:
# Create the pandas DataFrame from list
df = pd.DataFrame(classes, columns = ['name', 'label']) 
df.head()

In [ ]:
# transform class number from [1..196] to [0..195]
df['label'] = df['label']-1
df['label'].min(), df['label'].max()

In [ ]:
df.head()

In [ ]:
# verify number of unique classes
len(df['label'].unique())

In [ ]:
# verify length of car class annotations in cars_meta.mat
total_labels = len(class_annos["class_names"][0])
total_labels

In [ ]:
class_annos

In [ ]:
# extract class label strings into list
labels = [class_annos["class_names"][0][i][0] for i in range(total_labels)]
labels[:5], len(labels)

In [ ]:
# creating datafreame form list
df_label = pd.DataFrame(labels)
print(f' dataframe shape: {df_label.shape}')
df_label.head()

In [ ]:
# create data frame new_labels of class labels sorted according to class numbers
new_labels = pd.DataFrame(df_label[0][df['label']])

In [ ]:
new_labels.head()

In [ ]:
# reset index
new_labels.reset_index(inplace=True)
new_labels=new_labels.drop(columns='index', axis=1)
new_labels.head()

In [ ]:
# reaplace integer class numbers with string  class labels
df['label'] = new_labels
df.head()

In [ ]:
# create python data dictionary {}, path/file and image size(x,y)
# ps very long processing time
e_d = {k: PIL.Image.open(k).size for k in fnames}
#transform to list
row_sz, col_sz = list(zip(*e_d.values()))
# and to numpy
row_sz = np.array(row_sz); col_sz = np.array(col_sz)
row_sz.shape, col_sz.shape

In [ ]:
plt.hist(row_sz);

In [ ]:
plt.hist(col_sz);

In [ ]:
# histogram of all classes
# TODO

In [ ]:
# check for NoN values
df['label'].isnull().any()

In [ ]:
# split dataframe into train and test
import random
np.random.seed(42)

def splitDF(df, train_ratio):
    df['random_number'] = np.random.rand(len(df))

    train = df[df['random_number'] <= train_ratio]
    test = df[df['random_number'] > train_ratio]
    return (train, test)

In [ ]:
train, test = splitDF(df,0.8)

In [ ]:
len(train), len(test)

In [ ]:
train.head(10)

In [ ]:
test.head()

In [ ]:
Data_DIR = '/home/norvilr/ml/data/StanfordCars'
Model_DIR = '/home/norvilr/ml/github/AI/VSoAI/ImageClassification/StanfordCars2'

#create folders based on class labels and copy files
#run just once, to create folders and copy files

os.getcwd() 

def createClassFolders():
    for folder in df['label']:
        if not os.path.exists(folder):
            os.makedirs(folder)   

if os.getcwd()  != Data_DIR:
    os.chdir(Data_DIR)
!ls

if not os.path.exists('train'):
    os.makedirs('train')    
if not os.path.exists('test'):
    os.makedirs('test')
print(os.listdir())    

if os.path.exists('train'):
    os.chdir('train')
    createClassFolders()
if os.path.exists('../test'):
    os.chdir('../test')
    createClassFolders()    
os.chdir('../')    

print( 'created class subdirectories in train: ')
!find train -mindepth 1 -type d | wc -l
print( 'created class subdirectories in train: ')
!find test -mindepth 1 -type d | wc -l

#copy files into train subdirs
for index, row in train.iterrows():
    cmd = f'cp -f {row["name"]} train/"{row["label"]}"/'
    os.popen(cmd) 

#copy files into test subdirs
for index, row in test.iterrows():
    cmd = f'cp -f {row["name"]} test/"{row["label"]}"/'
    os.popen(cmd) 

#return to jupyter notebook directory
if os.getcwd()  != Model_DIR:
    os.chdir(Model_DIR)
!ls

In [ ]:
train_dir = 'train'
val_dir = 'test'

batch_size = 16

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    train_dir: transforms.Compose([
        transforms.RandomResizedCrop(224),
        #transforms.Resize(224),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    val_dir: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
classes = os.listdir(f'{Data_DIR}/train')
classes.sort()
ClassesNumer = len(classes)
print("Class Total Count: ", ClassesNumer)

In [ ]:

image_datasets = {x: datasets.ImageFolder(os.path.join(Data_DIR, x), transform=data_transforms[x]) for x in [train_dir, val_dir]}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in [train_dir, val_dir]}

dataset_sizes = {x: len(image_datasets[x]) for x in [train_dir, val_dir]}

class_names = image_datasets[train_dir].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
image_datasets[train_dir]

In [ ]:
# Iterate through the dataloader once
trainiter = iter(dataloaders[train_dir])
features, labels = next(trainiter)
features.shape, labels.shape

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in [train_dir, val_dir]:
            if phase == train_dir:
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == train_dir):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == train_dir:
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == val_dir and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Load a pretrained model and reset final fully connected layer
model_ft = models.resnet152(pretrained=True)
#model_ft = models.resnet34(pretrained=True)

#adding one fully connected layer with output=Classes
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, ClassesNumer)
model_ft = model_ft.to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)
#optimizer_ft = optim.Adam(model.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
exp_lr_scheduler = lr_scheduler.CyclicLR(optimizer_ft, base_lr=0.01, max_lr=0.1)


In [ ]:
# freeze all layers except last ones added next
for param in model_ft.parameters():
    param.requires_grad = False    
# Then unfreeze last classification layer only for feature extract
for param in model_ft.fc.parameters():
    param.requires_grad = True  
#To view which layers are freeze and which layers are not freezed:
for name, child in model_ft.named_children():
    for name_2, params in child.named_parameters():
        print(name_2, params.requires_grad)   

In [ ]:
# Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

In [ ]:
#UnFreeze all layers first
for param in model_ft.parameters():
    param.requires_grad = True 

# To view which layers are freeze and which layers are not freezed:
for name, child in model_ft.named_children():
    for name_2, params in child.named_parameters():
        print(name_2, params.requires_grad)

In [ ]:
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
exp_lr_scheduler = lr_scheduler.CyclicLR(optimizer_ft, base_lr=0.01, max_lr=1)

In [ ]:
# Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=21)

In [ ]:
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
exp_lr_scheduler = lr_scheduler.CyclicLR(optimizer_ft, base_lr=0.001, max_lr=0.1)
# Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=21)

In [ ]:
# Sequential(
#   (0): AdaptiveAvgPool2d(output_size=1)
#   (1): AdaptiveMaxPool2d(output_size=1)
#   (2): Flatten()
#   (3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#   (4): Dropout(p=0.25)
#   (5): Linear(in_features=1024, out_features=512, bias=True)
#   (6): ReLU(inplace)
#   (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#   (8): Dropout(p=0.5)
#   (9): Linear(in_features=512, out_features=10, bias=True)

In [ ]:
# Resize images from 224 to 299
data_transforms = {
    train_dir: transforms.Compose([
        transforms.RandomResizedCrop(299),
        #transforms.RandomRotation(degrees=15),
        #transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    val_dir: transforms.Compose([
        transforms.Resize(312),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
image_datasets = {x: datasets.ImageFolder(os.path.join(Data_DIR, x), transform=data_transforms[x]) for x in [train_dir, val_dir]}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=4) for x in [train_dir, val_dir]}


In [ ]:
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
exp_lr_scheduler = lr_scheduler.CyclicLR(optimizer_ft, base_lr=0.001, max_lr=0.1)
# Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=21)

In [ ]:
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
exp_lr_scheduler = lr_scheduler.CyclicLR(optimizer_ft, base_lr=0.0001, max_lr=0.01)
# Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=16)

In [ ]:
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0001, momentum=0.9)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
exp_lr_scheduler = lr_scheduler.CyclicLR(optimizer_ft, base_lr=0.00001, max_lr=0.001)
# Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)